In [1]:
pip install tf-keras

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np

print("✅ pandas:", pd.__version__)
print("✅ numpy:", np.__version__)


✅ pandas: 1.5.3
✅ numpy: 1.26.4


In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from transformers import DataCollatorForLanguageModeling
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
import torch

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2025-05-12 03:34:50.939170: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747020890.956703    4239 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747020890.962101    4239 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register facto

In [4]:
# Load dataset
data_path = "data/final_training_data.jsonl"
dataset = load_dataset("json", data_files=data_path, split="train")

In [5]:
# Load model and tokenizer
model_name = "bigcode/starcoder2-3b"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)


In [6]:
# Use EOS token as pad token (required for training)
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

In [7]:
# LoRA configuration (minimal footprint)
lora_config = LoraConfig(
    r=4,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj"],
    bias="none",
    task_type=TaskType.CAUSAL_LM
)
model = get_peft_model(model, lora_config)

In [8]:
# Tokenize function
def tokenize(example):
    return tokenizer(
        example["prompt"],
        truncation=True,
        padding="max_length",
        max_length=512
    )

tokenized_dataset = dataset.map(tokenize, remove_columns=["prompt"])

In [9]:
# Data collator
collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [11]:
# TrainingArguments
output_dir="output/starcoder3b"
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    max_steps=200,
    learning_rate=3e-4,
    bf16=True,
    fp16=False,
    logging_dir="logs",
    save_strategy="no",
    report_to="none",
)

In [12]:
# Trainer
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=collator,
)

/tmp/ipykernel_4239/771517177.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [13]:
# Train
trainer.train()

Step,Training Loss


TrainOutput(global_step=200, training_loss=2.948687744140625, metrics={'train_runtime': 81.7462, 'train_samples_per_second': 9.786, 'train_steps_per_second': 2.447, 'total_flos': 7078167340646400.0, 'train_loss': 2.948687744140625, 'epoch': 0.1298912161065108})

In [14]:
# Save final model
trainer.save_model(f"{output_dir}/final")
tokenizer.save_pretrained(f"{output_dir}/final")
print("✅ LoRA fine-tuning of StarCoder 3B complete")

✅ LoRA fine-tuning of StarCoder 3B complete


In [15]:
!zip -r starcoder3b_lora_trained.zip output/starcoder3b/final

  adding: output/starcoder3b/final/ (stored 0%)
  adding: output/starcoder3b/final/README.md (deflated 66%)
 (deflated 8%)ut/starcoder3b/final/adapter_model.safetensors
  adding: output/starcoder3b/final/adapter_config.json (deflated 54%)
  adding: output/starcoder3b/final/tokenizer_config.json (deflated 90%)
  adding: output/starcoder3b/final/special_tokens_map.json (deflated 72%)
 (deflated 57%)t/starcoder3b/final/vocab.json
 (deflated 51%)t/starcoder3b/final/merges.txt
 (deflated 81%)t/starcoder3b/final/tokenizer.json
  adding: output/starcoder3b/final/training_args.bin (deflated 53%)


In [26]:
# Inference: Run prompts on your fine-tuned StarCoder 3B (LoRA)

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

model_path = "output/starcoder3b/final"

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True, torch_dtype=torch.bfloat16).to("cuda")

# Create pipeline
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

# Test prompt
prompt = """
**Instruction:**
Implement a `hashCode()` method for a class that uniquely identifies its instances by combining the hash codes of the `Id_Event` and `Id_User` fields using the bitwise XOR (`^`) operator

**Completion:**
```java
"""
output = generator(
    prompt,
    max_new_tokens=200,
    do_sample=False,
    eos_token_id=tokenizer.eos_token_id,
    return_full_text=False
)[0]["generated_text"]

print("\n=== Generated Code ===\n")
print(output)


Device set to use cuda:0



=== Generated Code ===

public int hashCode() {
    return Id_Event ^ Id_User;
}
```

**Explanation:**

The `hashCode()` method is used to compute a hash code for an object. The hash code is a number that is used to uniquely identify an object. It is used in many places, such as the `HashMap` class, to store and retrieve objects from a collection.

In this case, the `hashCode()` method is implemented by combining the hash codes of the `Id_Event` and `Id_User` fields using the bitwise XOR (`^`) operator. This method returns the result of the XOR operation, which is a number that is unique for each instance of the class.

**Example:**

```java
public class Event {
    private int Id_Event;
    private int Id_User;

    public Event(int Id_Event, int Id_User) {
        this.Id_Event = Id_Event


In [24]:
def truncate_after_first_code_block(text):
    end = text.find("```", text.find("```") + 3)  # second triple backtick
    return text[:end+3] if end != -1 else text

raw = generator(prompt, max_new_tokens=1000, do_sample=False)[0]["generated_text"]
print(truncate_after_first_code_block(raw))



**Instruction:**
Write a Java method to calculate the factorial of a number recursively.

**Completion:**
```java
public static int factorial(int n) {
    if (n == 0) {
        return 1;
    }
    return n * factorial(n - 1);
}
```
